# ДЗ №3 
## Обучение моделей глубокого обучения на PyTorch

In [1]:
!pip3 install torch torchvision numpy matplotlib

Ваша задача на этой неделе - повторить модель трёхслойного перцептрона из прошолго задания на **PyTorch**, разобрать лучшие практики обучения моделей глубокого обучения и провести серию экспериментов

In [2]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

from typing import Tuple, List, Type, Dict, Any

Для того, чтобы эксперимент можно было повторить, хорошей практикой будет зафиксировать генератор случайных чисел. Также, рекоммендуется зафиксировать RNG в numpy и, если в качестве бэкенда используется cudnn - включить детерминированный режим.

Подробнее: https://pytorch.org/docs/stable/notes/randomness.html

In [3]:
torch.manual_seed(0)
np.random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

### Модель

Основным способом организации кода на **Pytorch** является модуль. Простые модели могут быть реализованны из готовых модулей ( к примеру, `torch.nn.Sequential`, `torch.nn.Linear` и т.д. ), для более сложных архитектур часто приходтся реализовывать собственные блоки. Это достаточно легко сделать - достаточно написать класс, наследуемый от `torch.nn.Module` и реализующий метод `.forward`, который принимает и возвращает тензоры ( `torch.Tensor` )

Пример реализации кастомного модуля из официальной документации: https://pytorch.org/tutorials/beginner/pytorch_with_examples.html3

#### Задание 1

Повторите реализацию трёхслойного перцептрона из предыдущего задания на **Pytorch**. Желательно также, чтобы реализация модели имела параметризуемую глубину ( количество слоёв ), количество параметров на каждом слое и функцию активации. Отсутствие такой возможности не снижает балл, но сильно поможет в освоении принципов построения нейросетей с применением библиотеки pytorch.

In [18]:
class Perceptron(torch.nn.Module):
    
    def __init__(self, 
                 input_resolution: Tuple[int, int] = (28, 28),
                 input_channels: int = 1, 
                 hidden_layer_features: List[int] = [256, 256, 256],
                 activation: Type[torch.nn.Module] = torch.nn.ReLU,
                 num_classes: int = 10,
                 n_hidden_neurons = 100):

        super(Perceptron, self).__init__()
        self.layer_1 = torch.nn.Linear(input_resolution[0] * input_resolution[1], n_hidden_neurons)
        self.act_1 = torch.nn.Sigmoid()
        self.layer_2 = torch.nn.Linear(n_hidden_neurons, num_classes)
        
    
    def forward(self, x):
        x = self.layer_1(x)
        x = self.act_1(x)
        x = self.layer_2(x)
        return x 

Заметим, что тут мы не считаем softmax на выходе, чтобы получить вероятности принадлежности к классам. Это делаем осознано, так как в pytorch есть функция CrossEntropyLoss, которая включает в себя взятие softmax и затем кросс-энтропии, но она более вычислительно стабильна и работает быстрее (так написано в документации)

Следующий код позволяет посмотреть архитектуру получившейся модели и общее количество обучаемых параметров. Мы хотим, чтобы количество параметров в модели было порядка сотен тысяч. Если у вас получается больше или меньше, попробуйте изменить архитектуру модели.

In [19]:
model = Perceptron()
print(model)
print('Total number of trainable parameters', 
      sum(p.numel() for p in model.parameters() if p.requires_grad))

Perceptron(
  (layer_1): Linear(in_features=784, out_features=100, bias=True)
  (act_1): Sigmoid()
  (layer_2): Linear(in_features=100, out_features=10, bias=True)
)
Total number of trainable parameters 79510


получили модель, у которой примерно 80000 параметров (условие, что порядка 100 000 выполнено)

### Обучающая выборка

На практике, наиболее важным для успеха обучения любой модели машинного обучения является этап подготовки данных. Модели глубокого обучения не являются исключением. Большая, чистая, репрезентативная и релевантная поставленной задаче обучающая выборка часто важнее, чем архитектура самой модели. В предлагаемой задаче используется качественный и проверенный временем MNIST. Однако в практических задачах часто будет получаться так, что лучшим способом добиться улучшения качества модели будет сбор дополнительных данных и очистка исходных данных.

### Предобработка данных

Для улучшения сходимости алгоритма обучения и качества полученной модели данные могут быть предварительно обработаны:

1. Среднее каждой входной переменной близко к нулю
2. Переменные отмасштабированы таким образом, что их дисперсии примерно одинаковы ( из соображений вычислительной устойчивости, мы хотим, чтобы все величины по порядку величины были близки к еденице )
3. По возможности, входные переменные не должны быть скоррелированны. Важнось этого пункта в последние годы ставится под сомнение, но всё-же в некоторых случаях это может влиять на результат

Подробнее можно почитать здесь: http://yann.lecun.com/exdb/publis/pdf/lecun-98b.pdf

### Аугментация (искусственное дополнение) обучающей выборки

В зависимости от задачи можно применять к признаковому описанию объектов обучающей выборки различные преобразования, которые позволят увеличить эффективный размер выборки без дополнительной разметки. К примеру, для задачи классификации кошек и собак можно зеркально отразить изображение вокруг вертикальной оси - при этом класс изображения не изменится, а само изображение останется по прежнему будет принадлежать исходному распределению. Есть много разных техник аугментации, и их применимость и эффективность сильно зависит от данных и задачи.

Подробнее можно почитать здесь: https://link.springer.com/content/pdf/10.1186/s40537-019-0197-0.pdf

### Задание 2

Обоснуйте, почему аугментация обучающей выборки позволяет добиться прироста качества модели, несмотря на то, что она не добавляет в неё дополнительную информацию.

#Your text here

### Задание 3

Какие осмысленные аугментации вы можете придумать для следующих наборов данных:

1. Набор изображений животных, размеченый на виды животных
2. Набор аудиозаписей голоса, размечеными на языки говорящего
3. Набор cо показаниями датчиков температуры, влажности и давления с одной из метеостанций, размеченый на признак наличия осадков

#Your text here

### Задание 4

Напишите пайплайн для предобработки и аугументации данных. В `torchvision.transforms` есть готовые реализации большинства распространённых техник. Если вы хотите добавить что-то своё, вы можете воспользоваться `torchvision.transforms.Lambda`. При этом следует понимать, что если нужно оценить качество модели на оригинальных данных, пайплайн предварительной обработки данных валидационной выборки не должен включать аугментаций. Следует помнить, однако, что существует подход аугментации данных в момент применения модели (test-time augmentation), который позволяет повысить качество модели в режиме исполнения.

Одним из обязательных шагов в вашем пайплайне должна быть конвертация данных в тензоры Pytorch (`torch.Tensor`): `torchvision.transforms.ToTensor()`.

In [20]:
train_transforms = torchvision.transforms.Compose([
                                                 
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (1.0,)),
    #transforms.Resize((28*28))
])

val_transforms = torchvision.transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (1.0,))
])

В трансформации данных переводим картинку в тензор, с помощью transforms.ToTensor() и нормализуем её


In [21]:
train_dataset = torchvision.datasets.MNIST(root='./mnist', 
                                           train=True, 
                                           download=True,
                                           transform=train_transforms)

val_dataset = torchvision.datasets.MNIST(root='./mnist', 
                                         train=False, 
                                         download=True, 
                                         transform=val_transforms)

In [22]:
print(train_dataset)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./mnist
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(1.0,))
           )


In [23]:
X_train = train_dataset.train_data # делаем данные для test и train для нашей нейронной сети
y_train = train_dataset.train_labels
X_test = val_dataset.test_data
y_test = val_dataset.test_labels

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:55: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:45: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [24]:
X_train.shape

torch.Size([60000, 28, 28])

In [25]:
X_train = X_train.reshape([-1, 28 * 28]) # переводим трёхмерный тензор в двумерный
X_test = X_test.reshape([-1, 28 * 28])

In [26]:
train_dataset.train_data.shape


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:55: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


torch.Size([60000, 28, 28])

In [27]:
X_train.shape

torch.Size([60000, 784])

Для начала просто преобразуем трёхмерный тензор, который поступает нам на вход, в двумерный (растянем картинку 28*28 в одну линию). Да, тем самымым мы потеряем информацию о том, какие пиксели находятся рядом, какие нет, но зато мы сможем применить наш многослойный Perceptron

Возможно это можно было сделать внутри класса torchvision.transforms.Compose, но я не разобралась как. Поэтому сделала это вне класса используя просто reshape, который подбирает первую зависимость автоматически, а вторую делает 28*28

Потом все равно использую dataloader, поэтому делаю reshape внутри каждой эпохи

Перед тем как запускать обучение всегда стоит посмотреть на данные после предобработки, и удостовериться, что они соответствуют ожидаемым

Не понимаю, как работает этот кусок кода...

In [ ]:
indices = np.random.randint(0, len(train_dataset), size=256)

fig, axes = plt.subplots(nrows=16, ncols=16, figsize=(32, 32))
for i, row in enumerate(axes):
    for j, ax in enumerate(row):
        sample_index = indices[i*16+j]
        sample, label = train_dataset[sample_index]
        #print(sample.cpu().numpy().transpose(1, 2, 0))
        ax.imshow(sample.cpu().numpy().transpose(1, 2, 0))
        ax.set_title(label)

### Обучение модели

Теперь, когда мы реализовали модель и подготовили данные мы можем приступить к непосредственному обучению модели. Костяк функции обучения написан ниже, далее вы должны будете реализовать ключевые части этого алгоритма

In [51]:
def train_model(model: torch.nn.Module, 
                train_dataset: torch.utils.data.Dataset,
                val_dataset: torch.utils.data.Dataset,
                loss_function: torch.nn.Module = torch.nn.CrossEntropyLoss(),
                optimizer_class: Type[torch.optim.Optimizer] = torch.optim,
                optimizer_params: Dict = {},
                initial_lr = 0.01,
                lr_scheduler_class: Any = torch.optim.lr_scheduler.ReduceLROnPlateau,
                lr_scheduler_params: Dict = {},
                batch_size = 64,
                max_epochs = 1000,
                early_stopping_patience = 20):
    optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr, **optimizer_params)
    lr_scheduler = lr_scheduler_class(optimizer, **lr_scheduler_params)
    
    train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)

    best_val_loss = None
    best_epoch = None
    loss_hist_train = []
    accuracy_hist_train = []

    loss_hist_val = []
    accuracy_hist_val = []
    for epoch in range(max_epochs):
        
        print(f'Epoch {epoch}')
        metric_train = train_single_epoch(model, optimizer, loss_function, train_loader)
        loss_hist_val.append(metric_train['loss'])
        accuracy_hist_val.append(metric_train['accuracy'])
        val_metrics = validate_single_epoch(model, loss_function, val_loader)
        loss_hist_train.append(val_metrics['loss'])
        accuracy_hist_train.append(val_metrics['accuracy'])
        print(f'Validation metrics: \n{val_metrics}')

        lr_scheduler.step(val_metrics['loss'])
        
        if best_val_loss is None or best_val_loss > val_metrics['loss']:
            print(f'Best model yet, saving')
            best_val_loss = val_metrics['loss']
            best_epoch = epoch
            torch.save(model, './best_model.pth')
            
        if epoch - best_epoch > early_stopping_patience:
            print('Early stopping triggered')
            return
    return loss_hist_train, accuracy_hist_train, loss_hist_val, accuracy_hist_val

### Задание 5

Реализуйте функцию, производящую обучение сети на протяжении одной эпохи ( полного прохода по всей обучающей выборке ). На вход будет приходить модель, оптимизатор, функция потерь и объект типа `DataLoader`. При итерировании по `data_loader` вы будете получать пары вида ( данные, целевая_переменная )

In [56]:
def train_single_epoch(model: torch.nn.Module,
                       optimizer: torch.optim.Optimizer, 
                       loss_function: torch.nn.Module, 
                       data_loader: torch.utils.data.DataLoader):
    loss_history = []
    accuracy_history = []
    for X, y in data_loader:
      X_batch = X.reshape([-1, 28 * 28])

      X_batch = X_batch.to(device) # переносим на видеопамять для вычисления (для скорости обучения делаю на colab)
      y = y.to(device)


      optimizer.zero_grad() # обнуляем градиент, так как он не обнуляется с предыдущей итерации

      pred = model.forward(X_batch) # получаю предсказание модели

      loss_value = loss_function(pred, y) # значение лосс функции на батче
      loss_history.append(loss_value)
      accuracy = (pred.argmax(dim=1) == y).float().mean() # точность это сколько из возможных угадала наша модель
      accuracy_history.append(accuracy)
      loss_value.backward() # производная по каждой из компонент нашей нейронной сети

      optimizer.step() # делаем шаг нашего оптимизатора
    return {'loss': float(sum(loss_history)/len(loss_history)), 'accuracy': float(sum(accuracy_history)/len(accuracy_history))}

### Задание 6

Реализуйте функцию производящую вычисление функции потерь на валидационной выборке.  На вход будет приходить модель, функция потерь и `DataLoader`. На выходе ожидается словарь с вида:
```
{
    'loss': <среднее значение функции потерь>,
    'accuracy': <среднее значение точности модели>
}
```

In [50]:
def validate_single_epoch(model: torch.nn.Module,
                          loss_function: torch.nn.Module, 
                          data_loader: torch.utils.data.DataLoader):
    loss_history = []
    accuracy_history = []
    accuracy = 0
    for X, y in data_loader:
      X_test = X.reshape([-1, 28 * 28])
      pred_test = model.forward(X_test)  # получаем результат на валидации

      loss_val = loss_function(pred_test, y) # лосс функция на валидации на одном батче
      loss_history.append(loss_val) #  добавляем в массив значение лосс функции на батче
      accuracy = (pred_test.argmax(dim=1) == y).float().mean() # точность это сколько из возможных угадала наша модель
      accuracy_history.append(accuracy)
      
    res_dict = {'loss': float(sum(loss_history)/len(loss_history)), 'accuracy': float(sum(accuracy_history)/len(accuracy_history))}
    return res_dict







Если вы корректно реализовали все предыдущие шаги и ваша модель имеет достаточное количество обучаемых параметров, то в следующей ячейке должен пойти процесс обучения, и мы должны достичь итоговой точности (в смысле меры accuracy, доли верных ответов) выше 90%

In [54]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') # если видеокарта доступно, будем переносить туда вычисления
model = model.to(device) # все параметры нейронной сети переносим на видеопамять, так как с ними вычисления происходят очень часто
# list(mnist_net.parameters()) # проверяем, что все веса и байесы переместились

In [52]:
train_model(model, 
            train_dataset=train_dataset, 
            val_dataset=val_dataset, 
            loss_function=torch.nn.CrossEntropyLoss(), 
            initial_lr=0.01)

Epoch 0
Validation metrics: 
{'loss': 0.16420666873455048, 'accuracy': 0.9523288011550903}
Best model yet, saving
Epoch 1
Validation metrics: 
{'loss': 0.17256665229797363, 'accuracy': 0.9499402642250061}
Epoch 2
Validation metrics: 
{'loss': 0.17492757737636566, 'accuracy': 0.9492436051368713}
Epoch 3
Validation metrics: 
{'loss': 0.15717710554599762, 'accuracy': 0.956409215927124}
Best model yet, saving
Epoch 4
Validation metrics: 
{'loss': 0.1613512635231018, 'accuracy': 0.9550158977508545}
Epoch 5
Validation metrics: 
{'loss': 0.1752701997756958, 'accuracy': 0.9487460255622864}
Epoch 6
Validation metrics: 
{'loss': 0.1733340322971344, 'accuracy': 0.9505374431610107}
Epoch 7
Validation metrics: 
{'loss': 0.16027899086475372, 'accuracy': 0.9541202187538147}
Epoch 8
Validation metrics: 
{'loss': 0.15008753538131714, 'accuracy': 0.9565087556838989}
Best model yet, saving
Epoch 9
Validation metrics: 
{'loss': 0.14529071748256683, 'accuracy': 0.9568073153495789}
Best model yet, saving
Ep

Выше представлено обучение модели, которая достигла точности 96.22


### Задание 7

Модифицируйте процесс обучения таким образом, чтобы достигнуть наилучшего качества на валидационной выборке. Модель должна оставаться N-слойным перцептроном с количеством обучаемых параметров <= 500000. Для обучения разрешается использовать только набор данных MNIST. Процесс обучения вы можете изменять по собственному усмотрению. К примеру, вы можете менять:

* Архитектуру модели в рамках наложенных ограничений на количество параметров и вид архитектуры (многослойный перцептрон)
* Функции активации в модели
* Используемый оптимизатор
* Расписание шага оптимизации
* Сэмплинг данных при обучении ( e.g. hard negative mining)

В результате мы ожидаем увидеть код экспериментов и любые инсайты, которые вы сможете получить в процессе

In [57]:
loss_hist_train, accuracy_hist_train, loss_hist_val, accuracy_hist_val = train_model(model, 
            train_dataset=train_dataset, 
            val_dataset=val_dataset, 
            loss_function=torch.nn.CrossEntropyLoss(), 
            initial_lr=0.01)

Epoch 0
Validation metrics: 
{'loss': 0.17392483353614807, 'accuracy': 0.9525278806686401}
Best model yet, saving
Epoch 1
Validation metrics: 
{'loss': 0.15757401287555695, 'accuracy': 0.9566082954406738}
Best model yet, saving
Epoch 2
Validation metrics: 
{'loss': 0.16630908846855164, 'accuracy': 0.9534235596656799}
Epoch 3
Validation metrics: 
{'loss': 0.15263965725898743, 'accuracy': 0.9589968323707581}
Best model yet, saving
Epoch 4
Validation metrics: 
{'loss': 0.1704322099685669, 'accuracy': 0.9541202187538147}
Epoch 5
Validation metrics: 
{'loss': 0.16922439634799957, 'accuracy': 0.9520302414894104}
Epoch 6
Validation metrics: 
{'loss': 0.1734420210123062, 'accuracy': 0.9532245397567749}
Epoch 7
Validation metrics: 
{'loss': 0.17181943356990814, 'accuracy': 0.9499402642250061}
Epoch 8
Validation metrics: 
{'loss': 0.15559880435466766, 'accuracy': 0.9575039744377136}
Epoch 9
Validation metrics: 
{'loss': 0.1765182465314865, 'accuracy': 0.9506369233131409}
Epoch 10
Validation metr

TypeError: ignored

In [58]:
plt.plot(loss_hist_train, label='loss function Test') # график убывания loss функции от номера эпохи
plt.plot(loss_hist_val, label='loss function validation', c='r')
plt.legend(loc='upper left')
plt.xlabel('$n\_epoxa$')
plt.ylabel('$loss$')

NameError: ignored

In [ ]:
plt.plot(accuracy_hist_val, c='r', label='accuracy_Test')
plt.plot(accuracy_hist_train, c='g', label='accuracy_Train')
plt.legend(loc='upper left')
plt.xlabel('$n\_epoxa$')
plt.ylabel('$accuracy$')